In [2]:
import dask_nested as dn # this should probably be nested_dask...
import nested_pandas as npd
import dask.dataframe as dd
import pandas as pd
from nested_pandas.series.dtype import NestedDtype
import pyarrow as pa

# How does Dask Work with `Nested-Pandas`?

In [5]:
# Toy Nested-Pandas Data
base_data = {"a": [1, 2, 3], "b": [2, 4, 6]}
base_index=[0,1,2]

base = npd.NestedFrame(data=base_data, index=base_index)

nested = pd.DataFrame(
    data={"c": [0, 2, 4, 1, 5, 3, 1, 4, 1], "d": [5, 4, 7, 5, 3, 1, 9, 3, 4],
         "label": ["a", "a", "b", "b", "a", "a", "b", "a" , "b"]},
    index=[0, 0, 0, 1, 1, 1, 2, 2, 2],
)
packed = npd.series.packer.pack_flat(nested, name="nested")
base = base.add_nested(nested, "nested")
base

,a,b,nested
0,1,2,c d label 0 0 5 a 1 2 4 a 2 4...
1,2,4,c d label 0 1 5 b 1 5 3 a 2 3...
2,3,6,c d label 0 1 9 b 1 4 3 a 2 1...


In [18]:
# Dask
dbase = dn.NestedFrame.from_nestedpandas(base)
dbase

,a,b,nested
npartitions=1,,,
0,int64,int64,"nested<c: [int64], d: [int64], label: [string]>"
2,...,...,...


In [17]:
# Some things work out of the box

dbase.query("nested.c>1").compute()

,a,b,nested
0,1,2,c d label 0 2 4 a 1 4 7 b
1,2,4,c d label 0 5 3 a 1 3 1 a
2,3,6,c d label 0 4 3 a


In [20]:
# map_partitions does seemingly give us a path for things that don't

dbase.map_partitions(lambda x: x.nested.nest.to_flat()).compute()

dtype nested<c: [int64], d: [int64], label: [string]>
<NestedExtensionArray>
[]
Length: 0, dtype: nested<c: [int64], d: [int64], label: [string]>


,c,d,label
0,0,5,a
0,2,4,a
0,4,7,b
1,1,5,b
1,5,3,a
1,3,1,a
2,1,9,b
2,4,3,a
2,1,4,b


In [13]:
from nested_pandas.utils import count_nested
dbase.map_partitions(lambda x: count_nested(x, "nested")).compute()

dtype nested<c: [int64], d: [int64], label: [string]>
<NestedExtensionArray>
[]
Length: 0, dtype: nested<c: [int64], d: [int64], label: [string]>


,a,b,nested,n_nested
0,1,2,c d label 0 0 5 a 1 2 4 a 2 4...,3
1,2,4,c d label 0 1 5 b 1 5 3 a 2 3...,3
2,3,6,c d label 0 1 9 b 1 4 3 a 2 1...,3
